# 本科毕业论文推荐算法对比实验

本实验旨在对比 **传统协同过滤推荐算法 (Traditional CF)** 与 **基于主题模型的协同过滤推荐算法 (Topic-CF, 本项目中采用 BERTopic)** 的推荐效果。

评价指标包括：
- **平均绝对误差 (MAE)**：衡量预测评分与实际评分的差异。值越小表示预测越准确。
- **准确率 (Precision)**、**召回率 (Recall)**、**F1-Score**：衡量 Top-N 推荐列表的准确程度。

In [ ]:
import os
import sys
import math
import time
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# 设置中文字体，防止 matplotlib 乱码
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 导入后端 Flask 应用的上下文，以便读取数据库
sys.path.append(os.path.abspath('../backend'))
from app import app
from models import db, User, Poem, Review
from recommendation_update import IncrementalRecommender
from bertopic_analysis import load_bertopic_model, get_document_vector

print("环境加载成功！")

## 1. 数据加载与预处理
从系统中读取所有的用户评论和打分数据。

In [ ]:
def load_data():
    with app.app_context():
        reviews = Review.query.all()
        data = []
        for r in reviews:
            # 默认如果没有rating则为3
            rating = r.rating if r.rating is not None else 3.0
            # 如果有liked点赞，稍微提高分值作为隐式反馈，更贴近实际喜爱偏好
            if r.liked:
                rating = min(5.0, rating + 1.0)
            data.append({
                'user_id': r.user_id,
                'poem_id': r.poem_id,
                'rating': rating
            })
        df = pd.DataFrame(data)
        
        # 过滤掉交互极少(小于2次)的用户，防止冷启动问题严重污染实验数据
        if len(df) > 0:
            user_counts = df['user_id'].value_counts()
            df = df[df['user_id'].isin(user_counts[user_counts >= 2].index)]
        
        return df

df = load_data()
print(f"加载了 {len(df)} 条有效交互数据")
df.head()

## 2. 划分训练集和测试集 (80% / 20%)

In [ ]:
if len(df) > 0:
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    print(f"训练集大小: {len(train_df)}, 测试集大小: {len(test_df)}")
else:
    print("没有足够的数据进行切分")
    train_df, test_df = pd.DataFrame(), pd.DataFrame()

# 转换为字典格式方便查询
train_data = defaultdict(dict)
for _, row in train_df.iterrows():
    train_data[int(row['user_id'])][int(row['poem_id'])] = float(row['rating'])
    
test_data = defaultdict(dict)
for _, row in test_df.iterrows():
    test_data[int(row['user_id'])][int(row['poem_id'])] = float(row['rating'])

## 3. 传统协同过滤 (User-CF)
仅利用用户历史评分计算用户相似度。

In [ ]:
def user_cf_predict(u, i, k=10):
    """预测用户 u 对物品 i 的评分 (传统 User-CF)"""
    # 计算当前用户的历史平均分
    u_mean = np.mean(list(train_data[u].values())) if u in train_data and train_data[u] else 3.0
    
    if u not in train_data:
        return u_mean
    
    sim_users = []
    for v in train_data:
        if v == u:
            continue
        if i not in train_data[v]:
            continue
        
        # 找共同评分的项目
        common_items = set(train_data[u].keys()).intersection(set(train_data[v].keys()))
        if not common_items:
            continue
        
        # 采用余弦相似度计算共同评分的相关性
        u_ratings = [train_data[u][item] for item in common_items]
        v_ratings = [train_data[v][item] for item in common_items]
        if sum(u_ratings) == 0 or sum(v_ratings) == 0:
            sim = 0
        else:
            sim = cosine_similarity([u_ratings], [v_ratings])[0][0]
        sim_users.append((v, sim))
        
    sim_users.sort(key=lambda x: x[1], reverse=True)
    sim_users = sim_users[:k]
    
    if not sim_users:
        return u_mean
    
    num = 0
    den = 0
    for v, sim in sim_users:
        if sim <= 0: continue
        v_mean = np.mean(list(train_data[v].values()))
        num += sim * (train_data[v][i] - v_mean)
        den += sim
        
    if den == 0:
        return u_mean
    
    pred = u_mean + num / den
    # 截断到 1-5 之间
    return max(1.0, min(5.0, pred))

## 4. 基于主题模型的协同过滤 (Topic-CF / BERTopic-CF)
利用提取的诗歌主题向量表示，计算用户画像向量，并在主题空间计算用户相似度。

In [ ]:
# 初始化现有系统中的 IncrementalRecommender，获取已有向量矩阵
recommender = IncrementalRecommender()
with app.app_context():
    recommender._build_poem_vector_matrix()
    
def get_user_topic_vector(user_id):
    """利用训练集数据构建用户主题向量 (同论文中的情感/偏好向量构建)"""
    if recommender.topic_matrix is None:
        return None
        
    history = train_data.get(user_id, {})
    if not history:
        return None
        
    user_vector = np.zeros(recommender.topic_matrix.shape[1])
    weight_sum = 0.0
    
    for poem_id, rating in history.items():
        poem_idx = recommender.poem_id_map.get(poem_id)
        if poem_idx is None:
            continue
            
        rating_weight = max(0.2, min(1.0, rating / 5.0))
        user_vector += recommender.topic_matrix[poem_idx] * rating_weight
        weight_sum += rating_weight
        
    if weight_sum > 0:
        user_vector /= weight_sum
    return user_vector

# 预计算所有用户的主题向量
user_topic_vectors = {}
for u in train_data.keys():
    vec = get_user_topic_vector(u)
    if vec is not None:
        user_topic_vectors[u] = vec

def topic_cf_predict(u, i, k=10):
    """预测用户 u 对物品 i 的评分 (基于主题空间的相似度)"""
    u_mean = np.mean(list(train_data[u].values())) if u in train_data and train_data[u] else 3.0
    
    if u not in train_data:
        return u_mean
        
    target_vec = user_topic_vectors.get(u)
    if target_vec is None:
        return u_mean
        
    sim_users = []
    for v, v_vec in user_topic_vectors.items():
        if v == u or v_vec is None:
            continue
        # 这里不需要共同评价过项目，因为是在主题向量空间直接计算用户全局相似度！
        # 但是仍然需要用户 v 对物品 i 有评分结果，才能进行预测
        if i not in train_data.get(v, {}):
            continue
            
        if np.sum(target_vec)==0 or np.sum(v_vec)==0:
            sim = 0
        else:
            sim = cosine_similarity([target_vec], [v_vec])[0][0]
        sim_users.append((v, sim))
        
    sim_users.sort(key=lambda x: x[1], reverse=True)
    sim_users = sim_users[:k]
    
    if not sim_users:
        return u_mean
        
    num = 0
    den = 0
    for v, sim in sim_users:
        if sim <= 0: continue
        v_mean = np.mean(list(train_data[v].values()))
        num += sim * (train_data[v][i] - v_mean)
        den += sim
        
    if den == 0:
        return u_mean
        
    pred = u_mean + num / den
    return max(1.0, min(5.0, pred))

## 5. 评价指标计算
针对测试集进行预测，计算总体 MAE，并模拟 Top-20 推荐计算 P/R/F1

In [ ]:
def evaluate_mae():
    cf_errors = []
    topic_cf_errors = []
    
    for u in test_data:
        for i, real_score in test_data[u].items():
            # 传统CF
            cf_pred = user_cf_predict(u, i)
            cf_errors.append(abs(cf_pred - real_score))
            
            # 主题CF
            topic_pred = topic_cf_predict(u, i)
            topic_cf_errors.append(abs(topic_pred - real_score))
            
    cf_mae = np.mean(cf_errors) if cf_errors else 0
    topic_mae = np.mean(topic_cf_errors) if topic_cf_errors else 0
    return cf_mae, topic_mae

def evaluate_top_n(N=20):
    cf_hits, cf_rec_count, cf_real_count = 0, 0, 0
    topic_hits, topic_rec_count, topic_real_count = 0, 0, 0
    
    all_items = list({item for user in train_data for item in train_data[user]})
    
    # 采样评估测试集中的用户
    sample_users = list(test_data.keys())
    
    for u in sample_users:
        real_positive_items = {i for i, r in test_data[u].items() if r >= 3.0}
        if not real_positive_items:
            continue
            
        cf_real_count += len(real_positive_items)
        topic_real_count += len(real_positive_items)
        
        # 只针对用户没有见过的项目进行推荐
        unseen_items = [i for i in all_items if i not in train_data.get(u, {})]
        
        # 为每个项目打分 
        cf_scores = {i: user_cf_predict(u, i) for i in unseen_items}
        topic_scores = {i: topic_cf_predict(u, i) for i in unseen_items}
        
        # 截取Top N
        cf_top_n = [item[0] for item in sorted(cf_scores.items(), key=lambda x:x[1], reverse=True)[:N]]
        topic_top_n = [item[0] for item in sorted(topic_scores.items(), key=lambda x:x[1], reverse=True)[:N]]
        
        cf_rec_count += N
        topic_rec_count += N
        
        cf_hits += len(set(cf_top_n).intersection(real_positive_items))
        topic_hits += len(set(topic_top_n).intersection(real_positive_items))
        
    def calc_metrics(hits, rec, real):
        if rec == 0 or real == 0:
            return 0, 0, 0
        p = hits / rec
        r = hits / real
        f1 = (2 * p * r) / (p + r) if (p + r) > 0 else 0
        return p, r, f1
        
    cf_p, cf_r, cf_f1 = calc_metrics(cf_hits, cf_rec_count, cf_real_count)
    topic_p, topic_r, topic_f1 = calc_metrics(topic_hits, topic_rec_count, topic_real_count)
    
    return (cf_p, cf_r, cf_f1), (topic_p, topic_r, topic_f1)

print("开始计算 MAE...")
cf_mae, topic_mae = evaluate_mae()
print("MAE 计算完成")

print("开始计算 Precision, Recall, F1...")
cf_metrics, topic_metrics = evaluate_top_n(N=20)
print("评价指标计算完成！")

## 6. 实验结果对比与可视化

In [ ]:
models = ['Traditional CF', 'Topic-CF (Proposed)']
mae_values = [cf_mae, topic_mae]
precision_values = [cf_metrics[0], topic_metrics[0]]
recall_values = [cf_metrics[1], topic_metrics[1]]
f1_values = [cf_metrics[2], topic_metrics[2]]

fig, axs = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('推荐算法效果对比实验结果 (Topic-CF vs Traditional CF)', fontsize=16)

colors = ['#1f77b4', '#ff7f0e']

# 1. MAE 图 (数值越小越好)
axs[0, 0].bar(models, mae_values, color=colors, width=0.4)
axs[0, 0].set_title('平均绝对误差 (MAE)对比 - 越低越好')
axs[0, 0].set_ylabel('MAE')
for i, v in enumerate(mae_values):
    axs[0, 0].text(i, v + 0.005, f"{v:.4f}", ha='center')

# 2. Precision 图
axs[0, 1].bar(models, precision_values, color=colors, width=0.4)
axs[0, 1].set_title('准确率 (Precision)对比 - 越高越好')
axs[0, 1].set_ylabel('Precision')
for i, v in enumerate(precision_values):
    axs[0, 1].text(i, v + 0.005, f"{v:.4f}", ha='center')

# 3. Recall 图
axs[1, 0].bar(models, recall_values, color=colors, width=0.4)
axs[1, 0].set_title('召回率 (Recall)对比 - 越高越好')
axs[1, 0].set_ylabel('Recall')
for i, v in enumerate(recall_values):
    axs[1, 0].text(i, v + 0.005, f"{v:.4f}", ha='center')

# 4. F1-Score 图
axs[1, 1].bar(models, f1_values, color=colors, width=0.4)
axs[1, 1].set_title('F1得分对比 - 越高越好')
axs[1, 1].set_ylabel('F1-Score')
for i, v in enumerate(f1_values):
    axs[1, 1].text(i, v + 0.005, f"{v:.4f}", ha='center')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

print("="*40)
print("实验结果摘要：")
print(f"传统 CF:")
print(f"  - MAE: {cf_mae:.4f}")
print(f"  - Precision: {cf_metrics[0]:.4f}")
print(f"  - Recall: {cf_metrics[1]:.4f}")
print(f"  - F1-Score: {cf_metrics[2]:.4f}")
print(f"基于主题的 CF:")
print(f"  - MAE: {topic_mae:.4f}")
print(f"  - Precision: {topic_metrics[0]:.4f}")
print(f"  - Recall: {topic_metrics[1]:.4f}")
print(f"  - F1-Score: {topic_metrics[2]:.4f}")
print("="*40)